# 1. Installing PySpark in Google Colab

In [34]:
#Tanmay Borundiya - 20220802277
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
39 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [36]:
# Creating a local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [37]:
# Loading data in pandas dataframe
import pandas as pd
amazon_df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/BDA/Amazon_Responded_Oct05.csv", on_bad_lines='skip')
amazon_df.head()

,id_str,tweet_created_at,user_screen_name,user_id_str,user_statuses_count,user_favourites_count,user_protected,user_listed_count,user_following,user_description,...,tweet_language,text_,favorite_count,favorited,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text
0,'793270689780203520',Tue Nov 01 01:57:25 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp Can you please DM me? A product I ...,0,False,AmazonHelp,NaN,85741735.0,0,False,NaN
1,'793281386912354304',Tue Nov 01 02:39:55 +0000 2016,AmazonHelp,85741735.0,2225450,11366,False,796,False,We answer Amazon support questions 7 days a we...,...,en,"@SeanEPanjab I'm sorry, we're unable to DM you...",0,False,SeanEPanjab,7.932707e+17,143515471.0,0,False,NaN
2,'793501578766319616',Tue Nov 01 17:14:53 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp It was purchased on https://t.co/g...,0,False,AmazonHelp,7.932814e+17,85741735.0,0,False,@AmazonHelp It was purchased on https://t.co/g...
3,'793501657346682880',Tue Nov 01 17:15:12 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,"@AmazonHelp I am following you now, if it help...",0,False,AmazonHelp,7.932814e+17,85741735.0,0,False,NaN
4,'793502854459879424',Tue Nov 01 17:19:57 +0000 2016,AmazonHelp,85741735.0,2225450,11366,False,796,False,We answer Amazon support questions 7 days a we...,...,en,@SeanEPanjab Please give us a call/chat so we ...,0,False,SeanEPanjab,7.935016e+17,143515471.0,0,False,@SeanEPanjab Please give us a call/chat so we ...


In [38]:
amazon_df.shape

(378132, 25)

In [39]:
# Dropping all null rows
amazon_df.dropna(how="all", inplace=True)
amazon_df.shape

(378132, 25)

In [40]:
# Replacing carriage return and new line characters with a space
amazon_df = amazon_df.replace({r'\r\n': ' '}, regex=True)
amazon_df.head()

,id_str,tweet_created_at,user_screen_name,user_id_str,user_statuses_count,user_favourites_count,user_protected,user_listed_count,user_following,user_description,...,tweet_language,text_,favorite_count,favorited,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text
0,'793270689780203520',Tue Nov 01 01:57:25 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp Can you please DM me? A product I ...,0,False,AmazonHelp,NaN,85741735.0,0,False,NaN
1,'793281386912354304',Tue Nov 01 02:39:55 +0000 2016,AmazonHelp,85741735.0,2225450,11366,False,796,False,We answer Amazon support questions 7 days a we...,...,en,"@SeanEPanjab I'm sorry, we're unable to DM you...",0,False,SeanEPanjab,7.932707e+17,143515471.0,0,False,NaN
2,'793501578766319616',Tue Nov 01 17:14:53 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp It was purchased on https://t.co/g...,0,False,AmazonHelp,7.932814e+17,85741735.0,0,False,@AmazonHelp It was purchased on https://t.co/g...
3,'793501657346682880',Tue Nov 01 17:15:12 +0000 2016,SeanEPanjab,143515471.0,51287,4079,False,74,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,"@AmazonHelp I am following you now, if it help...",0,False,AmazonHelp,7.932814e+17,85741735.0,0,False,NaN
4,'793502854459879424',Tue Nov 01 17:19:57 +0000 2016,AmazonHelp,85741735.0,2225450,11366,False,796,False,We answer Amazon support questions 7 days a we...,...,en,@SeanEPanjab Please give us a call/chat so we ...,0,False,SeanEPanjab,7.935016e+17,143515471.0,0,False,@SeanEPanjab Please give us a call/chat so we ...


In [42]:
# Converting Pandas Dataframe into Spark Dataframe
amazon_df = amazon_df.astype(str) # Converting pandas df to string first
amazon_sdf = spark.createDataFrame(amazon_df)
amazon_sdf.show(10, False) # False allows us to show entire content of the columns

+--------------------+------------------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------------+--------------------+------------------+------------------------------+--------------+------------------------------------------------------------------------------------------------------------------------------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |tweet_created_at              |user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_followin

In [43]:
# Columns in df
amazon_sdf.columns

['id_str',
 'tweet_created_at',
 'user_screen_name',
 'user_id_str',
 'user_statuses_count',
 'user_favourites_count',
 'user_protected',
 'user_listed_count',
 'user_following',
 'user_description',
 'user_location',
 'user_verified',
 'user_followers_count',
 'user_friends_count',
 'user_created_at',
 'tweet_language',
 'text_',
 'favorite_count',
 'favorited',
 'in_reply_to_screen_name',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id_str',
 'retweet_count',
 'retweeted',
 'text']

In [44]:
# Schema: Datatypes associated with columns
amazon_sdf.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- tweet_created_at: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_statuses_count: string (nullable = true)
 |-- user_favourites_count: string (nullable = true)
 |-- user_protected: string (nullable = true)
 |-- user_listed_count: string (nullable = true)
 |-- user_following: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_friends_count: string (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- tweet_language: string (nullable = true)
 |-- text_: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- favorited: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |--

In [45]:
# Total number of rows
amazon_sdf.count()

378132

# Step 2: Extracting the three columns

In [46]:
# Extracting columns 'user_id_str', 'user_followers_count', and 'text_'
amazon_sub_df = amazon_sdf.select(amazon_sdf.user_id_str, amazon_sdf.user_followers_count.cast('int').alias('user_followers_count'), amazon_sdf.text_)
amazon_sub_df.show(20, False)

+-----------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id_str|user_followers_count|text_                                                                                                                                                        |
+-----------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|143515471.0|1503                |@AmazonHelp Can you please DM me? A product I ordered last year never arrived.                                                                               |
|85741735.0 |149569              |@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL                          |
|143515471.0|1503                |@

In [47]:
# Checking datatype of columns
amazon_sub_df.printSchema()

root
 |-- user_id_str: string (nullable = true)
 |-- user_followers_count: integer (nullable = true)
 |-- text_: string (nullable = true)



In [48]:
# Number of distinct users
import pyspark.sql.functions as f
amazon_sub_df.select(f.countDistinct("user_id_str")).show()

+---------------------------+
|count(DISTINCT user_id_str)|
+---------------------------+
|                      71533|
+---------------------------+



# Step 3: Removing the duplicated records

In [49]:
# Checking number of rows (tweets) for a particular user
amazon_sub_df.filter(amazon_sub_df.user_id_str == '85741735.0').count()

170690

In [50]:
# Removing duplicate records by keeping just the maximum number of followers for any user
# Step 1: First let us get the max followers for every user
import pyspark.sql.functions as f
maxf = amazon_sub_df.groupBy("user_id_str").agg(f.max("user_followers_count").alias("max")).alias("maxf")
maxf.show()

+------------+-----+
| user_id_str|  max|
+------------+-----+
| 182037498.0|   38|
|  16669765.0|   57|
| 264218502.0|   70|
|1873374613.0|  174|
|1305569118.0|  128|
|3255474476.0|    1|
|  29207516.0|  213|
| 123058781.0|  380|
|1079912580.0|  673|
| 277017869.0| 1103|
| 553031167.0|  491|
|  43597889.0| 1915|
|  45659327.0|  338|
| 101381642.0|  602|
|2811064347.0|11588|
|2325986832.0|  141|
| 998124780.0|  102|
| 220146294.0|  497|
|1362528962.0|  567|
| 112242731.0|   45|
+------------+-----+
only showing top 20 rows



In [51]:
# Total number of rows
maxf.count()

71533

In [52]:
# Step 2: Let us now join this with the original df (amazon_sub_df) to get all the rows of users which match their max follower count obtained in 'maxf'
from pyspark.sql.functions import col
amazon_sub_df = amazon_sub_df.alias("amazon_sub_df") # defining alias for original df
amazon_sub_df2 = amazon_sub_df.join(maxf, (col("user_followers_count") == col("max")) &
                                    (col("amazon_sub_df.user_id_str") == col("maxf.user_id_str"))).select(
                                     col("amazon_sub_df.user_id_str"), col("amazon_sub_df.user_followers_count"), col("amazon_sub_df.text_"))

In [53]:
amazon_sub_df2.show(20, False)

+------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id_str |user_followers_count|text_                                                                                                                                           |
+------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|94245683.0  |1085                |Been having issues sleeping lately due to increased anxiety about why the heck my background check isn't finished for Amazon yet.               |
|61218672.0  |719                 |@AmazonHelp how can I get an update on an order which has been delayed? My tracking details say it's been delivered &amp;  haven't been updated |
|61218672.0  |719                 |@AmazonHelp I received an email to say delivery was delayed 

In [54]:
# Total number of rows
amazon_sub_df2.count()

194626

In [55]:
# Checking number of rows (tweets) for that user again
amazon_sub_df2.filter(amazon_sub_df2.user_id_str == '85741735.0').count()

13

# Step 4: Finding popular users

In [56]:
# Creating a filter to find popular users who have more than 5000 followers
popular_df = amazon_sub_df2.filter(amazon_sub_df2.user_followers_count >= 5000)
popular_df.count() # number of rows

5063

In [57]:
popular_df.show(20, False)

+------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id_str |user_followers_count|text_                                                                                                                                                 |
+------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|97610175.0  |15738               |@amazonIN Order# 171-7573811-5583505 has not delivered. Almost 25 days gone. How many months do you need more?                                        |
|97610175.0  |15738               |@AmazonHelp What I have to fill could you tell me more clearly?                                                                                       |
|97610175.0  |15738               |@amazonIN @AmazonHelp I didn't

In [58]:
# Check: Sorting follower count Ascending
popular_df.sort(popular_df.user_followers_count).show(20, False)

+---------------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|user_id_str          |user_followers_count|text_                                                                                                                                          |
+---------------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|26359772.0           |5003                |@AmazonHelp any chance you can tell me what time American Gods is going online on May 1st?                                                     |
|213353996.0          |5007                |@AmazonHelp Had two shipments marked as "delivered" yesterday that never appeared. Still marked as delivered and I have 0/6 items.             |
|213353996.0          |5007                |@AmazonHelp

In [59]:
# Check: Sorting follower count Descending
popular_df.sort((popular_df.user_followers_count).desc()).show(20, False)

+------------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id_str |user_followers_count|text_                                                                                                                                                     |
+------------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|20793816.0  |2776204             |@Liberal_Lisa You're all set. We'll remind you when #PrimeDay is here! https://t.co/kQdsFAczL6                                                            |
|83554234.0  |1958810             |@AmazonHelp Thank you. message sent                                                                                                                       |
|83554234.0  |1958810             |@AmazonHel

In [60]:
# Number of distinct users
popular_df.select('user_id_str').distinct().count()

2130

In [61]:
# Finding number of tweets per user using groupBy
groupedUsers = popular_df.groupby('user_id_str').count().withColumnRenamed("count","tweet_count")

In [62]:
# Sorting in descending order of count
groupedUsers.sort((groupedUsers.tweet_count).desc()).show(20)

+------------+-----------+
| user_id_str|tweet_count|
+------------+-----------+
|  17535421.0|         23|
|2454614098.0|         20|
|  16789906.0|         18|
|2950894703.0|         18|
| 518389533.0|         17|
| 568744096.0|         15|
| 153426866.0|         14|
|  22486545.0|         14|
|  17225620.0|         14|
|  85741735.0|         13|
|   2551781.0|         13|
|  27326436.0|         12|
|  15658980.0|         11|
|  60964525.0|         11|
|  17742722.0|         11|
|  31697976.0|         11|
| 313202447.0|         11|
|  14247313.0|         11|
|  74177248.0|         11|
| 274852927.0|         11|
+------------+-----------+
only showing top 20 rows



# Step 5: Finding top words and save the results

In [63]:
# Counting words frequency of the tweets posted by the popular users from step 4
# Reading column 'text_' from spark df (popular_df) to a python list, which will then be read into RDD object
tweet = popular_df.select("text_").rdd.flatMap(lambda x: x).collect()
tweet[0:5]

['@amazonIN Order# 171-7573811-5583505 has not delivered. Almost 25 days gone. How many months do you need more?',
 '@AmazonHelp What I have to fill could you tell me more clearly?',
 "@amazonIN @AmazonHelp I didn't ask for a gift voucher, I asked you to deliver my product ordered. #ShameonyouAmazon",
 "@AmazonHelp Don't regret, try to deliver the product ordered at the earliest. #ShameonyouAmazon",
 "Looks like my parcel isn't coming today @AmazonHelp 😒 I live 5 mins from a warehouse how can it be late?"]

In [64]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [65]:
# Reading 'tweet' to a RDD object using spark context
tweet_rdd = sc.parallelize(tweet)
tweet_rdd.take(5)

['@amazonIN Order# 171-7573811-5583505 has not delivered. Almost 25 days gone. How many months do you need more?',
 '@AmazonHelp What I have to fill could you tell me more clearly?',
 "@amazonIN @AmazonHelp I didn't ask for a gift voucher, I asked you to deliver my product ordered. #ShameonyouAmazon",
 "@AmazonHelp Don't regret, try to deliver the product ordered at the earliest. #ShameonyouAmazon",
 "Looks like my parcel isn't coming today @AmazonHelp 😒 I live 5 mins from a warehouse how can it be late?"]

In [66]:
# Function for cleaning the text of tweets
import string
import re

def clean_tweet(x):

  # Delete all the URLs in the tweets
  text00 = re.sub(r'www\S+', '', x)
  text01 = re.sub(r'http\S+', '', text00)

  # Delete all the numbers in the tweets
  text1 = ''.join([i for i in text01 if not i.isdigit()])

  # Delete all the punctuation marks in the tweets
  text2 = text1.translate(str.maketrans('','',string.punctuation))

  # Convert text to LOWERCASE
  text3 = text2.lower()

  return text3

In [67]:
# Cleaning the text of tweets: 1. Removing URLs, 2. Removing non-alphabets, 3. Lowercase
clean_tweet_rdd = tweet_rdd.map(clean_tweet)
clean_tweet_rdd.take(10)

['amazonin order  has not delivered almost  days gone how many months do you need more',
 'amazonhelp what i have to fill could you tell me more clearly',
 'amazonin amazonhelp i didnt ask for a gift voucher i asked you to deliver my product ordered shameonyouamazon',
 'amazonhelp dont regret try to deliver the product ordered at the earliest shameonyouamazon',
 'looks like my parcel isnt coming today amazonhelp 😒 i live  mins from a warehouse how can it be late',
 'amazonhelp it says it was dispatched on the th and should arrive today but nothing when something i ordered after this arrived on sat',
 'amazonhelp hi ive had money go out of my account and says its for amazon prime which i have never signed up for',
 'amazonhelp it says i can end it but not cancel will this work',
 'is it true that amazon no longer sends out citystate info to those who purchase items off wish lists 🤔 ive bee told its a new feature',
 'amazonhelp anyoneanywhere online i can get an update on a parcel paid t

In [68]:
# Building Map function
map = clean_tweet_rdd.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
map.take(5)

[('amazonin', 1), ('order', 1), ('', 1), ('has', 1), ('not', 1)]

In [69]:
# Building Reduce function
counts = map.reduceByKey(lambda a, b: a + b)
counts.take(5)

[('', 3239), ('almost', 29), ('days', 179), ('how', 244), ('months', 42)]

In [70]:
# Total number of distinct words
print(len(counts.collect()))

6855


In [71]:
# Getting the Top 10 most popular words and their words frequency
# Sorting 'counts' in descending order and getting the first 10 elements
countsSortedTopTen = counts.takeOrdered(10, lambda a: -a[1] if len(a[0]) > 0 else False) # Conditioned on that number of characters in the string should be at least 1
countsSortedTopTen

[('amazonhelp', 3647),
 ('i', 2896),
 ('to', 2491),
 ('the', 2356),
 ('a', 1616),
 ('it', 1551),
 ('my', 1482),
 ('amazon', 1434),
 ('and', 1379),
 ('you', 1185)]

In [72]:
# Writing list to a text file
with open("/content/drive/MyDrive/Colab_Notebooks/BDA/2277_tanmay.txt", 'w') as f:
    for item in countsSortedTopTen:
        f.write(item[0] + ', ' + str(item[1]) + '\n')